In [1]:
#Project3 

In [ ]:
# List of commonly shortcuts that can be used in Jupyter notebook 
# Shift + Enter: Run the cell # Esc + A : Insert a cell above the current cell 
# Esc + B : Insert a cell below the current cell 
# Ctrl + / : Comment the selected code 
# Shift + Ctrl + - : Break the cell at the cursor position 

# Python Project - To Predict Counterfeit Sales 

## Method Used - Regression
## Models Used - RandomForest , Xgboost
## Target Variable - 'Counterfeit_Sales'
## Language Used/IDE - Jupyter Notebook
## Aim - Score=1-(MAE/1660) --- Score should be greater than 0.5

In [2]:
#Importing Libraries 
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [3]:
#Reading Data

cf_train=pd.read_csv(r'C:\Users\Monica\Desktop\Projects\Python Projects 1\Project 3\Data_for_Project_3\counterfeit_train.csv')
cf_test=pd.read_csv(r'C:\Users\Monica\Desktop\Projects\Python Projects 1\Project 3\Data_for_Project_3\counterfeit_test.csv')

In [4]:
#Making a copy
cf_test1 = cf_test.copy()

In [5]:
#Understanding Data
cf_train.head(3)

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920


In [6]:
cf_train.columns

Index(['Medicine_ID', 'Counterfeit_Weight', 'DistArea_ID', 'Active_Since',
       'Medicine_MRP', 'Medicine_Type', 'SidEffect_Level',
       'Availability_rating', 'Area_Type', 'Area_City_Type', 'Area_dist_level',
       'Counterfeit_Sales'],
      dtype='object')

In [7]:
cf_test.columns  # each column with names

Index(['Medicine_ID', 'Counterfeit_Weight', 'DistArea_ID', 'Active_Since',
       'Medicine_MRP', 'Medicine_Type', 'SidEffect_Level',
       'Availability_rating', 'Area_Type', 'Area_City_Type',
       'Area_dist_level'],
      dtype='object')

In [8]:
## cf_test does not have the outcome variable - in order to combine the two we will add the response variable to cf_test
cf_test['Counterfeit_Sales']=np.nan

In [9]:
# combine the two datasets to pre-process the data together since the data on which the model is trained should undergo the same preprocessing as the data on which the predictions are made

In [10]:
#Combine data
cf_train['data']='train'
cf_test['data']='test'
cf_test=cf_test[cf_train.columns]   # the columns in the two data frames should be in the same order to enable concatenation
cf_all=pd.concat([cf_train,cf_test],axis=0)

In [11]:
cf_all.shape

(8523, 13)

In [12]:
cf_all.dtypes

Medicine_ID             object
Counterfeit_Weight     float64
DistArea_ID             object
Active_Since             int64
Medicine_MRP           float64
Medicine_Type           object
SidEffect_Level         object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Area_dist_level         object
Counterfeit_Sales      float64
data                    object
dtype: object

In [13]:
list(zip(cf_all.columns,cf_all.dtypes,cf_all.nunique()))

[('Medicine_ID', dtype('O'), 1557),
 ('Counterfeit_Weight', dtype('float64'), 415),
 ('DistArea_ID', dtype('O'), 10),
 ('Active_Since', dtype('int64'), 9),
 ('Medicine_MRP', dtype('float64'), 5970),
 ('Medicine_Type', dtype('O'), 16),
 ('SidEffect_Level', dtype('O'), 2),
 ('Availability_rating', dtype('float64'), 7884),
 ('Area_Type', dtype('O'), 4),
 ('Area_City_Type', dtype('O'), 3),
 ('Area_dist_level', dtype('O'), 4),
 ('Counterfeit_Sales', dtype('float64'), 3142),
 ('data', dtype('O'), 2)]

In [14]:
cf_all.head()

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales,data
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026,train
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520,train
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920,train
3,GWC40,11.800,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130,train
4,QMN13,NaN,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402,train


In [15]:
#Data Pre-processing
#Analyze each feature(drop,convert to numeric, create dummies)

In [16]:
# DisArea_ID,Counterfeit_Weight,Medicine_MRP,Availability_rating,Counterfeit_Sales: to numeric
# Area_dist_level,Medicine_ID,Active since : drop
#Area_City_Type,Area_Type : replace with 1,2..
#Medicine_Type,SidEffect_Level : Dummies

In [17]:
cf_all.drop(['Active_Since','Medicine_ID','Area_dist_level'],axis=1,inplace=True)

In [18]:
k=cf_all['DistArea_ID'].str.split("0",expand=True)

In [19]:
k.head()

,0,1,2
0,Area,46,None
1,Area,27,None
2,Area,46,None
3,Area,46,None
4,Area,19,None


In [20]:
cf_all['distAr_ID']=pd.to_numeric(k[1])

In [21]:
cf_all['distAr_ID'].isnull().sum()

0

In [22]:
cf_all.drop(['DistArea_ID'],axis=1,inplace=True)

In [23]:
cf_all['distAr_ID'].head()

0    46
1    27
2    46
3    46
4    19
Name: distAr_ID, dtype: int64

In [24]:
# Creating dummies using .get_dummies function
cf_all['SideEffect_level']=pd.get_dummies(cf_all['SidEffect_Level'],drop_first=True,prefix='SideEffect')

In [25]:
del cf_all['SidEffect_Level'] # del is the command used to delete column in dataset
cf_all['SideEffect_level']=cf_all.SideEffect_level.astype(float)  # Converting column into float type

In [26]:
cf_all.head() # checking data in tabular form using .head()

,Counterfeit_Weight,Medicine_MRP,Medicine_Type,Availability_rating,Area_Type,Area_City_Type,Counterfeit_Sales,data,distAr_ID,SideEffect_level
0,13.100,160.2366,Antimalarial,0.070422,DownTown,Tier 1,1775.5026,train,46,0.0
1,NaN,110.4384,Mstablizers,0.013000,CityLimits,Tier 3,3069.1520,train,27,1.0
2,9.025,259.4092,Cardiac,0.060783,DownTown,Tier 1,2603.0920,train,46,1.0
3,11.800,99.9830,OralContraceptives,0.065555,DownTown,Tier 1,1101.7130,train,46,1.0
4,NaN,56.4402,Hreplacements,0.248859,MidTownResidential,Tier 1,158.9402,train,19,0.0


In [27]:
cf_all.dtypes  # dtypes will display data-type of each column

Counterfeit_Weight     float64
Medicine_MRP           float64
Medicine_Type           object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Counterfeit_Sales      float64
data                    object
distAr_ID                int64
SideEffect_level       float64
dtype: object

In [28]:
K=cf_all['Area_City_Type'].str.split(" ",expand=True)  # splitting string 

In [29]:
K[1].head()

0    1
1    3
2    1
3    1
4    1
Name: 1, dtype: object

In [30]:
cf_all['City_type']=pd.to_numeric(K[1]) # converting to numeric 

In [31]:
cf_all.drop(['Area_City_Type'],axis=1,inplace=True) # droping/removing a column

In [32]:
cf_all.head()

,Counterfeit_Weight,Medicine_MRP,Medicine_Type,Availability_rating,Area_Type,Counterfeit_Sales,data,distAr_ID,SideEffect_level,City_type
0,13.100,160.2366,Antimalarial,0.070422,DownTown,1775.5026,train,46,0.0,1
1,NaN,110.4384,Mstablizers,0.013000,CityLimits,3069.1520,train,27,1.0,3
2,9.025,259.4092,Cardiac,0.060783,DownTown,2603.0920,train,46,1.0,1
3,11.800,99.9830,OralContraceptives,0.065555,DownTown,1101.7130,train,46,1.0,1
4,NaN,56.4402,Hreplacements,0.248859,MidTownResidential,158.9402,train,19,0.0,1


In [33]:
print(cf_all['Area_Type'].unique())       # display each unique value for that column
print(cf_all['Area_Type'].nunique())      # display total number of unique values for that column

['DownTown' 'CityLimits' 'MidTownResidential' 'Industrial']
4


In [34]:
ArType=pd.get_dummies(cf_all['Area_Type'],drop_first=True,prefix='AreaType')  # crating dummies for mentioned column


In [35]:
ArType.head()

,AreaType_DownTown,AreaType_Industrial,AreaType_MidTownResidential
0,1,0,0
1,0,0,0
2,1,0,0
3,1,0,0
4,0,0,1


In [36]:
cf_all=pd.concat([cf_all,ArType],axis=1)
del cf_all['Area_Type']

In [37]:
MedType=pd.get_dummies(cf_all['Medicine_Type'],drop_first=True,prefix='MedType')
cf_all=pd.concat([cf_all,MedType],axis=1)
del cf_all['Medicine_Type']

In [38]:
#Check missing values and impute it

In [39]:
cf_all.isnull().sum()   #ignore target variable null values

Counterfeit_Weight             1463
Medicine_MRP                      0
Availability_rating               0
Counterfeit_Sales              1705
data                              0
distAr_ID                         0
SideEffect_level                  0
City_type                         0
AreaType_DownTown                 0
AreaType_Industrial               0
AreaType_MidTownResidential       0
MedType_Antacids                  0
MedType_Antibiotics               0
MedType_Antifungal                0
MedType_Antimalarial              0
MedType_Antipyretics              0
MedType_Antiseptics               0
MedType_Antiviral                 0
MedType_Cardiac                   0
MedType_Hreplacements             0
MedType_Mstablizers               0
MedType_MuscleRelaxants           0
MedType_OralContraceptives        0
MedType_Statins                   0
MedType_Stimulants                0
MedType_Tranquilizers             0
dtype: int64

In [40]:
#cf_all.loc[cf_all['Counterfeit_Sales'].isnull(),'Counterfeit_Sales']=np.mean(cf_all['Counterfeit_Sales'])

In [41]:
cf_all.loc[cf_all['Counterfeit_Weight'].isnull(),'Counterfeit_Weight']=np.mean(cf_all['Counterfeit_Weight']) 

In [42]:
cf_all['Counterfeit_Weight'].isnull().sum()

0

In [43]:
# Data Pre-processing is complete and data is in expected format

In [ ]:
#Let's separate our two data sets and remove the unnecessary columns that we added while combining them.
# Separate the train and test data 
# We will break the train data further into train and validation datasets 

In [44]:
#Separate data
ld_train=cf_all[cf_all['data']=='train']
del ld_train['data']
ld_test=cf_all[cf_all['data']=='test']
ld_test.drop(['Counterfeit_Sales','data'], axis=1,inplace=True)

C:\Users\Monica\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
ld_train.shape, ld_test.shape

((6818, 25), (1705, 24))

In [46]:
#we broke the train data further into train and validation datasets to enable us to assess the models performance 
# Instead, you can use cross validation to get the best parameters for the model and assess model performance

In [47]:
#Split the data (Splitting training data into train and validation)
cf_train, cf_test = train_test_split(ld_train, test_size=0.2,random_state=2)

In [48]:
cf_train.shape,cf_test.shape # display number of rows and columns present in dataset

((5454, 25), (1364, 25))

In [49]:
x_train=cf_train.drop('Counterfeit_Sales',axis=1)
y_train=cf_train['Counterfeit_Sales']
x_test=cf_test.drop('Counterfeit_Sales',axis=1)
y_test=cf_test['Counterfeit_Sales']

In [50]:
#Model Implementation
#Let's build a model on training data.

In [ ]:
# Random Forest

In [51]:
regressor=RandomForestRegressor()
#n_estimators=100, random_state=0,

In [52]:
param_dist={'n_estimators' : [50,100,500,1000],
            'max_features' : [5,10,15],
            #'min_samples_split' : [25,50,100],
            #'min_samples_leaf': [None,5,10,15],
            #'max_depth': [None,50,150,250,350,450,550]
            'max_depth':[1,2,3,4,5,6]
            }

In [53]:
n_iter_search=10
random_search=RandomizedSearchCV(regressor,param_distributions=param_dist,
                                 n_iter=n_iter_search,
                                 scoring='neg_mean_absolute_error',
                                 cv=5,
                                 verbose = 5,
                                 n_jobs=-1)

In [54]:
search = random_search.fit(x_train,y_train )

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   49.4s finished


In [55]:
random_model = search.best_estimator_

In [56]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [57]:
report(search.cv_results_,5)

Model with rank: 1
Mean validation score: -763.778 (std: 25.58967)
Parameters: {'n_estimators': 100, 'max_features': 15, 'max_depth': 5}

Model with rank: 2
Mean validation score: -766.283 (std: 20.92756)
Parameters: {'n_estimators': 50, 'max_features': 10, 'max_depth': 6}

Model with rank: 3
Mean validation score: -825.990 (std: 22.02692)
Parameters: {'n_estimators': 500, 'max_features': 10, 'max_depth': 4}

Model with rank: 4
Mean validation score: -842.879 (std: 22.98795)
Parameters: {'n_estimators': 100, 'max_features': 15, 'max_depth': 3}

Model with rank: 5
Mean validation score: -889.482 (std: 19.67514)
Parameters: {'n_estimators': 500, 'max_features': 10, 'max_depth': 3}



In [58]:
pred = random_model.predict(x_test)

In [59]:
pred.shape

(1364,)

In [60]:
mean_absolute_error(y_test,pred)

774.0405973307908

In [61]:
mae = mean_absolute_error(y_test,pred)

In [62]:
mae

774.0405973307908

In [63]:
score1=(1-mae/1660)
print(score1)

0.5337104835356682


In [ ]:
# Check the performance of the model on the validation data set 

In [64]:
test_pred = random_model.predict(ld_test)

In [65]:
ld_test.shape, test_pred.shape

((1705, 24), (1705,))

In [66]:
#pd.DataFrame(test_pred).to_csv("Monica_Desai_P3_part2.csv")

In [67]:
#regressor.fit(x_train,y_train)

In [68]:
#v_test=regressor.predict(x_test)

In [69]:
#Second Model Implementation - This is done to check if performance improves or not 

In [ ]:
# Xgboost 

In [70]:
#anaconda search -t conda xgboost( in Anaconda Prompt)

In [71]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [72]:
import xgboost as xgb

In [73]:
from xgboost.sklearn import XGBRegressor

In [74]:
model=xgb.XGBRegressor()

In [75]:
model.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [76]:
xgpredict=model.predict(x_test)

In [ ]:
# We will now compare our predictions to the actual values in the validation dataset 
# this will let us know how good our model is 
# There are multiple ways to do this; we will use mean absolute error here 

In [77]:
mae_xg = mean_absolute_error(y_test,xgpredict)
print('Mean Absolute Error : ',mae_xg)

Mean Absolute Error :  833.2019808855146


In [78]:
mae = mean_absolute_error(y_test,pred)

In [79]:
mae

774.0405973307908

In [80]:
score1=(1-mae/1660)
print(score1) # asked in aim

0.5337104835356682


In [81]:
param_dist = {
              "max_depth": [2,3,4,5,6],
              "learning_rate":[0.01,0.05,0.1,0.3,0.5],
    "min_child_weight":[4,5,6],
              "subsample":[i/10.0 for i in range(6,10)],
 "colsample_bytree":[i/10.0 for i in range(6,10)],
               "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
              "gamma":[i/10.0 for i in range(0,5)],
    "n_estimators":[100,500,700,1000],
    'scale_pos_weight':[2,3,4,5,6,7,8,9]
    
              }

In [82]:
from sklearn.model_selection import RandomizedSearchCV

In [83]:
n_iter=10

random_search=RandomizedSearchCV(model,n_jobs=-1,verbose=2,cv=10,n_iter=n_iter,
                                 param_distributions=param_dist)

In [84]:
random_search.fit(x_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.5min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          gpu_id=-1, importance_type='gain',
                                          interaction_constraints='',
                                          learning_rate=0.300000012,
                                          max_delta_step=0, max_depth=6,
                                          min_child_weight=1, missing=nan,
                                          monotone_constraints='()',...
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.3,
                                                          0.5],
             

In [85]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [86]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.603 (std: 0.02240)
Parameters: {'subsample': 0.6, 'scale_pos_weight': 4, 'reg_alpha': 100, 'n_estimators': 500, 'min_child_weight': 4, 'max_depth': 4, 'learning_rate': 0.01, 'gamma': 0.3, 'colsample_bytree': 0.7}

Model with rank: 2
Mean validation score: 0.601 (std: 0.02277)
Parameters: {'subsample': 0.9, 'scale_pos_weight': 8, 'reg_alpha': 100, 'n_estimators': 1000, 'min_child_weight': 6, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.9}

Model with rank: 3
Mean validation score: 0.592 (std: 0.02459)
Parameters: {'subsample': 0.9, 'scale_pos_weight': 8, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 6, 'max_depth': 4, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.9}

Model with rank: 4
Mean validation score: 0.584 (std: 0.02540)
Parameters: {'subsample': 0.9, 'scale_pos_weight': 8, 'reg_alpha': 0.1, 'n_estimators': 500, 'min_child_weight': 4, 'max_depth': 2, 'learning_rate': 0.1, 'gamma': 0.

In [87]:
xbg_best=xgb.XGBRegressor(subsample= 0.9, scale_pos_weight= 6, reg_alpha= 0.01, 
                          n_estimators= 100, min_child_weight= 6, max_depth= 3, learning_rate= 0.05, 
                          gamma= 0.3, colsample_bytree= 0.9)

In [88]:
xbg_best.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.3, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=6, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0.01,
             reg_lambda=1, scale_pos_weight=6, subsample=0.9,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [89]:
a=xbg_best.predict(x_test)

In [90]:
mae_xgb_best = mean_absolute_error(y_test,a)
print('Mean Absolute Error : ',mae_xgb_best)

Mean Absolute Error :  758.0086709752684


In [91]:
score1=(1-mae_xgb_best/1660)
print(score1)

0.5433682704968263


In [92]:
# After applying model1 (RF) - score was 0.534 and after applying model2 (Xgboost) - score is 0.543 so yes there is improvement (you can try different models if you want) but i stop here as my business objective is achieved which was to score more than 0.50

In [93]:
test_pred=xbg_best.predict(ld_test)

In [ ]:
# Saving and submitting result in the format that they have mentioned for.

In [94]:
submission=pd.DataFrame(list(zip(cf_test1['Medicine_ID'],list(test_pred))),
                       columns=['Medicine_ID','Counterfeit_Sales'])

In [95]:
submission.to_csv('Submission_Monica_Desai_Counterfeit_Sales_Predict.csv',index=False)